In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

YOLO

In [ ]:
!curl -L "https://universe.roboflow.com/ds/faNhoDkUzx?key=Ech4EH0Ufp" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
import os
import torch
print(torch.cuda.is_available())
!pip install easyocr
!pip install ultralytics
import ultralytics
ultralytics.checks()

predict foto / video

In [ ]:
!yolo task=detect mode=predict model=/content/gdrive/MyDrive/PlatNomorTest/model/YOLO3/best.pt conf=0.50 source=/content/test/images/

In [ ]:
!yolo task=detect mode=predict model=/content/gdrive/MyDrive/PlatNomorTest/model/YOLO3/best.pt conf=0.50 source=/content/gdrive/MyDrive/PlatNomorTest/video/

In [7]:
from ultralytics import YOLO
import cv2
import easyocr

dict_angka = {
    '0' : 'O',
    '1' : 'I',
    '2' : 'Z',
    '3' : 'B',
    '4' : 'A',
    '5' : 'S',
    '6' : 'G',
    '8' : 'B'
}


import string
def convert_huruf(angka):
  if angka in dict_angka:
    angka = dict_angka[angka]
  return angka

def cek_plat(text):
  itung_angka = 0;
  kalimat = ''
  for i in range(0, len(text)):
    test = text[i]
    if(itung_angka == 4):
      test = convert_huruf(text[i])
    elif(itung_angka <= 3 and text[i].isnumeric() == True):
      itung_angka += 1
    kalimat += test
  return kalimat


model = YOLO("/ITERA/TUGAS AKHIR/PLAT NOMOR/Hasil 2/MODELS/YOLOV9/weights/best.pt")
def predict(model, img, classes = [],conf =0.5):
  if classes:
    results = model.predict(img, classes = classes, conf = conf)
  else:
    results = model.predict(img, conf = conf)

  return results

def predict_detect(model, img, classes = [],conf =0.5, rect_thick=2, text_thick=1):
  results = predict(model, img, classes, conf=conf)
  for result in results:
    for box in result.boxes:
      cv2.rectangle(img, (int(box.xyxy[0][0]), int(box.xyxy[0][1])),
                    (int(box.xyxy[0][2]), int(box.xyxy[0][3])), (255, 0, 0), rect_thick)

      text = ''
      x1 = int(box.xyxy[0][0])
      y1 = int(box.xyxy[0][1])
      x2 = int(box.xyxy[0][2])
      y2 = int(box.xyxy[0][3])
      cropped_image = img[y1:y2, x1:x2]
      cropped_image = cv2.cvtColor(img[y1:y2, x1:x2], cv2.COLOR_BGR2GRAY)
      reader = easyocr.Reader(['id'])


      hasil = reader.readtext(cropped_image, width_ths = 10000, ycenter_ths = 0.50, slope_ths = 100
                              , allowlist = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
                              , height_ths = 250, decoder = 'wordbeamsearch', beamWidth = 10) #CUMA PLAT NOMOR


      text = ''
      print(hasil) #######
      if(hasil == []):
        text = 'Null'
      else:
        #print(hasil) ########
        '''
        count = 0
        while(count < 3 and count < len(hasil) ):
          text = text + hasil[count][1]
          count +=1
          text = cek_plat(text)
          text = text + " "
        '''

        text = text + hasil[0][1]
        text = text.upper().replace(' ', '')
        text = cek_plat(text)


      color = (0,0,0)
      fontScale = 0.8
      font = cv2.FONT_HERSHEY_SIMPLEX
      org = (x1,y1-18)
      thickness = 2
      (text_w, text_h), baseline = cv2.getTextSize(text, font, fontScale, thickness)
      text_color_bg = (255,255,255)
      cv2.rectangle(img, org, (x1 + text_w, y1 - text_h - baseline - 12), text_color_bg, -1)
      img = cv2.putText(img, text, org, font,
                      fontScale, color, thickness, cv2.LINE_AA)
      cv2.rectangle(img, (int(box.xyxy[0][0]), int(box.xyxy[0][1] - 15)),
                    (int(box.xyxy[0][2]), int(box.xyxy[0][1]) ), (255, 0, 0), -1)
      cv2.putText(img, f"{result.names[int(box.cls[0])]}" + "%.5s" % f"{box.conf[0]}",
                        (int(box.xyxy[0][0]+5), int(box.xyxy[0][1]) - 2 ),
                        cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), text_thick)
  return img

In [ ]:
img = cv2.imread('/ITERA/TUGAS AKHIR/PLAT NOMOR/DATASET GAMBAR TEST 123/plat(1).jpg')
result_img = predict_detect(model, img, classes=[],conf=0.5)
cv2.imshow('image',result_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import glob

no_test = 1

for imageName in glob.glob('/content/test/images/*jpg'):
  img = cv2.imread(imageName)
  result_img, _ = predict_detect(model, img, classes=[],conf=0.5)
  imageName2 = 'pred' + str(no_test) + '.jpg'
  cv2.imwrite(imageName2, result_img)
  no_test = no_test + 1

In [ ]:
import os
import glob
import shutil

test_gambar = '/content/'
test_output = '/content/hasil2/'

gambar = glob.glob(os.path.join(test_gambar, '*.jpg'), recursive = True)
print(gambar)
for x in gambar:
  dst_path = os.path.join(test_output, os.path.basename(x))
  shutil.move(x, dst_path)

In [ ]:
!zip -r /content/donlod.zip /content/hasil2

In [ ]:
cap = cv2.VideoCapture('/ITERA/TUGAS AKHIR/PLAT NOMOR/VIDEO CCTV/EDIT/video3.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
'''
hasil123 = cv2.VideoWriter('test.mp4',
                         cv2.VideoWriter_fourcc(*'mp4v'),
                         15, size) #nomor = fps
'''
if (cap.isOpened()== False):
  print("Error opening video stream or file")
(success, img) = cap.read()
while(success):
  result_img = predict_detect(model, img, classes=[], conf=0.5)
  #hasil123.write(result_img)
  cv2.imshow("Image", result_img)
  test123 = cv2.waitKey(1)
  if test123 == 27:
    break
  (success, img) = cap.read()
cap.release()
#hasil123.release()
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture(0)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
size = (frame_width, frame_height)
'''
hasil123 = cv2.VideoWriter('test.mp4',
                         cv2.VideoWriter_fourcc(*'mp4v'),
                         15, size) #nomor = fps
'''
if (cap.isOpened()== False):
  print("Error opening video stream or file")
(success, img) = cap.read()
while(success):
  result_img = predict_detect(model, img, classes=[], conf=0.5)
  #hasil123.write(result_img)
  cv2.imshow("Image", result_img)
  test123 = cv2.waitKey(1)
  if test123 == 27:
    break
  (success, img) = cap.read()
cap.release()
#hasil123.release()
cv2.destroyAllWindows()

Eval OCR

In [ ]:
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
import cv2
import easyocr

def count_characters(string):
    character_count = {}

    for char in string:
        if char in character_count:
            character_count[char] += 1
        else:
            character_count[char] = 1

    return character_count

def compare_character_counts(string1, string2):
    count1 = count_characters(string1)
    count2 = count_characters(string2)

    all_chars = set(count1.keys()).union(set(count2.keys()))

    comparison = {}
    for char in all_chars:
        comparison[char] = (count1.get(char, 0), count2.get(char, 0))

    return comparison

dict_angka = {
    '0' : 'O',
    '1' : 'I',
    '2' : 'Z',
    '3' : 'B',
    '4' : 'A',
    '5' : 'S',
    '6' : 'G',
    '8' : 'B'
}


import string
def convert_huruf(angka):
  if angka in dict_angka:
    angka = dict_angka[angka]
  return angka

def cek_plat(text):
  itung_angka = 0;
  kalimat = ''
  for i in range(0, len(text)):
    test = text[i]
    if(itung_angka == 4):
      test = convert_huruf(text[i])
    elif(itung_angka <= 3 and text[i].isnumeric() == True):
      itung_angka += 1
    kalimat += test
  return kalimat


model = YOLO("/content/gdrive/MyDrive/PlatNomorTest/model/YOLOv9/best.pt")
def predict(model, img, classes = [],conf =0.5):
  if classes:
    results = model.predict(img, classes = classes, conf = conf)
  else:
    results = model.predict(img, conf = conf)

  return results

def eval(model, img, classes = [],conf =0.5, rect_thick=2, text_thick=1):
  results = predict(model, img, classes, conf=conf)
  for result in results:
    for box in result.boxes:
      cv2.rectangle(img, (int(box.xyxy[0][0]), int(box.xyxy[0][1])),
                    (int(box.xyxy[0][2]), int(box.xyxy[0][3])), (255, 0, 0), rect_thick)

      text = ''
      x1 = int(box.xyxy[0][0])
      y1 = int(box.xyxy[0][1])
      x2 = int(box.xyxy[0][2])
      y2 = int(box.xyxy[0][3])
      cropped_image = img[y1:y2, x1:x2]
      cropped_image = cv2.cvtColor(img[y1:y2, x1:x2], cv2.COLOR_BGR2GRAY)
      reader = easyocr.Reader(['id'])


      hasil = reader.readtext(cropped_image, width_ths = 10000, ycenter_ths = 0.50, slope_ths = 100
                              , allowlist = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
                              , height_ths = 250, decoder = 'wordbeamsearch', beamWidth = 10) #CUMA PLAT NOMOR


      text = ''
      #print(hasil) #######
      if(hasil == []):
        text = 'null'
      else:
        #print(hasil) ########
        '''
        count = 0
        while(count < 3 and count < len(hasil) ):
          text = text + hasil[count][1]
          count +=1
          text = cek_plat(text)
          text = text + " "
        '''

        text = text + hasil[0][1]
        text = text.upper().replace(' ', '')
        text = cek_plat(text)

      return text

In [ ]:
import glob
cm_tp=0;
cm_tn=0;
cm_fp=0;
cm_fn=0;
for imageName in glob.glob('/content/gdrive/MyDrive/PlatNomorTest/TestOCR/*.jpg'):
  test123 = '.jpg'
  stringName = (imageName.replace(test123, '')) + '.txt'

  f = open(stringName, "r")
  text2 = f.read()
  #print(text2)

  img = cv2.imread(imageName)
  text1 = eval(model, img, classes=[],conf=0.5)

  #print(text1)

  character_comparison = compare_character_counts(text2, text1)
  '''
  for char, counts in character_comparison.items():
    print(f"'{char}': String1 = {counts[0]}, String2 = {counts[1]}")
  '''
  for char,counts in character_comparison.items():
    if len(text2) == len(text1):
      if counts[0] == counts[1]:
        cm_tp += counts[1]
      elif counts[0] > counts[1]:
        cm_tp += counts[1]
        cm_fn += counts[0] - counts[1]
    elif len(text2) > len(text1):
      if counts[0] == counts[1]:
        cm_tp += counts[1]
      elif counts[0] > counts[1]:
        cm_tp += counts[1]
        cm_fn += counts[0] - counts[1]
    elif len(text2) < len(text1):
      cm_fp += len(text1) - len(text2)
      if counts[0] == counts[1]:
        cm_tp += counts[1]
      elif counts[0] > counts[1]:
        cm_tp += counts[1]
        cm_fn += counts[0] - counts[1]

precision = cm_tp/(cm_tp+cm_fp)
recall = cm_tp/(cm_tp+cm_fn)
f1_score = 2 * (precision * recall) / (precision + recall)

print("True Positive : ",cm_tp)
print("False Positive : ",cm_fp)
print("False Negative : ",cm_fn)
print("True Negative : ",cm_tn)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)

test

In [ ]:
#!yolo val model=/content/gdrive/MyDrive/PlatNomorTest/model/YOLO3/best.pt data=/content/data.yaml split=test conf=0.5

test2

In [ ]:
import ultralytics
from ultralytics import YOLO
model = YOLO("/content/gdrive/MyDrive/PlatNomorTest/model/YOLO3/best.pt")
metrics = model.val(data="/content/data.yaml", imgsz=640, batch=16, conf=0.5, device="0", save_json=True, split='test')

zip

In [ ]:
#kalau utf-8 error
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from google.colab import files
!zip -r /content/donlod.zip /content/runs

pindahin ke drive

In [ ]:
!cp donlod.zip /content/gdrive/MyDrive/PlatNomorTest

Mask R-CNN

In [ ]:
!curl -L "https://universe.roboflow.com/ds/seOB03nXfn?key=M8PBpdsehF" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
!pip install pyyaml==5.1
#https://detectron2.readthedocs.io/tutorials/install.html
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
!pip show detectron2

In [ ]:
import detectron2
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2 import model_zoo

register_coco_instances("my_dataset_train", {}, "/content/train/_annotations.coco.json", "/content/train")
register_coco_instances("my_dataset_val", {}, "/content/valid/_annotations.coco.json", "/content/valid")
register_coco_instances("my_dataset_test", {}, "/content/test/_annotations.coco.json", "/content/test")

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
cfg.MODEL.MASK_ON = False
cfg.MODEL.DEVICE='cpu' #pake CPU only
cfg.MODEL.WEIGHTS = "/content/gdrive/MyDrive/PlatNomorTest/model/Mask640/model_final.pth"

In [ ]:
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "my_dataset_val")
inference_on_dataset(predictor.model, val_loader, evaluator)

In [ ]:
!pip install easyocr

In [ ]:
import easyocr
import cv2
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode
import glob
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog
from google.colab.patches import cv2_imshow

dict_angka = {
    '0' : 'O',
    '1' : 'I',
    '2' : 'Z',
    '3' : 'B',
    '4' : 'A',
    '5' : 'S',
    '6' : 'G',
    '8' : 'B'
}


import string
def convert_huruf(angka):
  if angka in dict_angka:
    angka = dict_angka[angka]
  return angka

def cek_plat(text):
  itung_angka = 0;
  kalimat = ''
  for i in range(0, len(text)):
    test = text[i]
    if(itung_angka == 4):
      test = convert_huruf(text[i])
    elif(itung_angka <= 3 and text[i].isnumeric() == True):
      itung_angka += 1
    kalimat += test
  return kalimat


test_metadata = MetadataCatalog.get("my_dataset_train")
dataset_dicts = DatasetCatalog.get("my_dataset_train")

no_test = 1
predictor = DefaultPredictor(cfg)

for imageName in glob.glob('/content/test/*jpg'):
  img = cv2.imread(imageName)
  outputs = predictor(img)


  pred_img = predictor(img)
  kotak = pred_img["instances"].pred_boxes
  x1,x2,y1,y2 = 0,0,0,0
  for i in kotak.__iter__():
    x1 = int(i.cpu().numpy()[0])
    y1 = int(i.cpu().numpy()[1])
    x2 = int(i.cpu().numpy()[2])
    y2 = int(i.cpu().numpy()[3])
    cropped_image = img[y1:y2, x1:x2]
    cropped_image = cv2.cvtColor(img[y1:y2, x1:x2], cv2.COLOR_BGR2GRAY)
    reader = easyocr.Reader(['id'])
    result = reader.readtext(cropped_image, width_ths = 10000, ycenter_ths = 0.50, slope_ths = 100
                              , allowlist = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
                              , height_ths = 250, decoder = 'wordbeamsearch', beamWidth = 10) #CUMA PLAT NOMOR



    text = ''
    if(result == []):
      text = 'Null'
    else:
      text = text + result[0][1]
      text = text.upper().replace(' ', '')
      text = cek_plat(text)

    color = (0,0,0)
    fontScale = 0.8
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (x1,y1-5)
    thickness = 2
    (text_w, text_h), baseline = cv2.getTextSize(text, font, fontScale, thickness)
    text_color_bg = (255,255,255)
    cv2.rectangle(img, org, (x1 + text_w + 10 - baseline, y1 - text_h - baseline), text_color_bg, -1)
    img = cv2.putText(img, text, org, font,
                    fontScale, color, thickness, cv2.LINE_AA)


  v = Visualizer(img[:, :, ::-1],
                metadata=test_metadata,
                scale=0.8
                 )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])

  imageName2 = 'pred' + str(no_test) + '.jpg'
  cv2.imwrite(imageName2, out.get_image()[:, :, ::-1])
  no_test = no_test + 1


In [ ]:
import os
import glob
import shutil

test_gambar = '/content/'
test_output = '/content/output/'

gambar = glob.glob(os.path.join(test_gambar, '*.jpg'), recursive = True)
print(gambar)
for x in gambar:
  dst_path = os.path.join(test_output, os.path.basename(x))
  shutil.move(x, dst_path)

In [ ]:
!zip -r /content/donlod.zip /content/output

Predict per gambar

In [ ]:
import cv2
predictor = DefaultPredictor(cfg)
image_path = '/content/pred1.jpg'
img = cv2.imread(image_path)
outputs = predictor(img)
v = Visualizer(img[:, :, ::-1],
              metadata=test_metadata,
              scale=0.8
              )
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

Predict video

In [ ]:
import easyocr
import cv2
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode
import glob
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog
from google.colab.patches import cv2_imshow
import time

dict_angka = {
    '0' : 'O',
    '1' : 'I',
    '2' : 'Z',
    '3' : 'B',
    '4' : 'A',
    '5' : 'S',
    '6' : 'G',
    '8' : 'B'
}


import string
def convert_huruf(angka):
  if angka in dict_angka:
    angka = dict_angka[angka]
  return angka

def cek_plat(text):
  itung_angka = 0;
  kalimat = ''
  for i in range(0, len(text)):
    test = text[i]
    if(itung_angka == 4):
      test = convert_huruf(text[i])
    elif(itung_angka <= 3 and text[i].isnumeric() == True):
      itung_angka += 1
    kalimat += test
  return kalimat

predictor = DefaultPredictor(cfg)
cap = cv2.VideoCapture('/content/gdrive/MyDrive/PlatNomorTest/video/video6.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
hasil123 = cv2.VideoWriter('test.mp4',
                         cv2.VideoWriter_fourcc(*'mp4v'),
                         15, size)

if (cap.isOpened()== False):
  print("Error opening video stream or file")

(success, img) = cap.read()
while(success):
  start_time = time.time()
  pred_img = predictor(img)
  end_time = time.time()
  inference_time = end_time - start_time
  print(f"Inference time: {inference_time:.4f} seconds")
  kotak = pred_img["instances"].pred_boxes
  x1,x2,y1,y2 = 0,0,0,0
  for i in kotak.__iter__():
    x1 = int(i.cpu().numpy()[0])
    y1 = int(i.cpu().numpy()[1])
    x2 = int(i.cpu().numpy()[2])
    y2 = int(i.cpu().numpy()[3])
    cropped_image = img[y1:y2, x1:x2]
    cropped_image = cv2.cvtColor(img[y1:y2, x1:x2], cv2.COLOR_BGR2GRAY)
    reader = easyocr.Reader(['id'])
    hasil = reader.readtext(cropped_image, width_ths = 10000, ycenter_ths = 0.50, slope_ths = 100
                              , allowlist = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
                              , height_ths = 250, decoder = 'wordbeamsearch', beamWidth = 10)
    text = ''
    if(hasil == []):
      text = 'Null'
    else:
      text = text + hasil[0][1]
      text = text.upper().replace(' ', '')
      text = cek_plat(text)

    color = (0,0,0)
    fontScale = 0.8
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (x1,y1-5)
    thickness = 2
    (text_w, text_h), baseline = cv2.getTextSize(text, font, fontScale, thickness)
    text_color_bg = (255,255,255)
    cv2.rectangle(img, org, (x1 + text_w + 10 - baseline, y1 - text_h - baseline), text_color_bg, -1)
    img = cv2.putText(img, text, org, font,
                    fontScale, color, thickness, cv2.LINE_AA)


  v = Visualizer(img,
                metadata={},
                )
  output = v.draw_instance_predictions(predictions["instances"].to("cpu"))
  #cv2_imshow(output.get_image()[:, :, ::-1])
  hasil123.write(output.get_image())

  key = cv2.waitKey(1)
  if key == ord("t"):
    break
  (success, img) = cap.read()

cap.release()
hasil123.release()
cv2.destroyAllWindows()

print("The video was successfully saved")

In [ ]:
!cp test.avi /content/gdrive/MyDrive/PlatNomorTest

single image

In [ ]:
import easyocr
import cv2
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode
import glob
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog
from google.colab.patches import cv2_imshow

test_metadata = MetadataCatalog.get("my_dataset_train")
dataset_dicts = DatasetCatalog.get("my_dataset_train")

predictor = DefaultPredictor(cfg)
image_path = '/content/plat(398).jpg'
img = cv2.imread(image_path)
outputs = predictor(img)

pred_img = predictor(img)
kotak = pred_img["instances"].pred_boxes
x1,x2,y1,y2 = 0,0,0,0

for i in kotak.__iter__():
  x1 = int(i.cpu().numpy()[0])
  y1 = int(i.cpu().numpy()[1])
  x2 = int(i.cpu().numpy()[2])
  y2 = int(i.cpu().numpy()[3])
  cropped_image = img[y1:y2, x1:x2]
  cropped_image = cv2.cvtColor(img[y1:y2, x1:x2], cv2.COLOR_BGR2GRAY)
  reader = easyocr.Reader(['id'])
  result = reader.readtext(cropped_image)

  if(result == []):
    text = 'Null'
  else:
    text = result[0][1]
    text = text.upper().replace(' ', '')

  color = (0,0,0)
  fontScale = 0.8
  font = cv2.FONT_HERSHEY_SIMPLEX
  org = (x1,y1-5)
  thickness = 2
  (text_w, text_h), baseline = cv2.getTextSize(text, font, fontScale, thickness)
  text_color_bg = (255,255,255)
  cv2.rectangle(img, org, (x1 + text_w - baseline, y1 - text_h - baseline), text_color_bg, -1)
  img = cv2.putText(img, text, org, font,
                   fontScale, color, thickness, cv2.LINE_AA)

v = Visualizer(img[:, :, ::-1],
              metadata=test_metadata,
              scale=0.8
              )

out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

cv2_imshow(out.get_image()[:, :, ::-1])

EVAL


In [ ]:
import easyocr
import cv2
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode
import glob
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog
from google.colab.patches import cv2_imshow

def count_characters(string):
    character_count = {}

    for char in string:
        if char in character_count:
            character_count[char] += 1
        else:
            character_count[char] = 1

    return character_count

def compare_character_counts(string1, string2):
    count1 = count_characters(string1)
    count2 = count_characters(string2)

    all_chars = set(count1.keys()).union(set(count2.keys()))

    comparison = {}
    for char in all_chars:
        comparison[char] = (count1.get(char, 0), count2.get(char, 0))

    return comparison

dict_angka = {
    '0' : 'O',
    '1' : 'I',
    '2' : 'Z',
    '3' : 'B',
    '4' : 'A',
    '5' : 'S',
    '6' : 'G',
    '8' : 'B'
}


import string
def convert_huruf(angka):
  if angka in dict_angka:
    angka = dict_angka[angka]
  return angka

def cek_plat(text):
  itung_angka = 0;
  kalimat = ''
  for i in range(0, len(text)):
    test = text[i]
    if(itung_angka == 4):
      test = convert_huruf(text[i])
    elif(itung_angka <= 3 and text[i].isnumeric() == True):
      itung_angka += 1
    kalimat += test
  return kalimat

no_test = 1
predictor = DefaultPredictor(cfg)

import glob
cm_tp=0;
cm_tn=0;
cm_fp=0;
cm_fn=0;

for imageName in glob.glob('/content/gdrive/MyDrive/PlatNomorTest/TestOCR/*.jpg'):
  test123 = '.jpg'
  stringName = (imageName.replace(test123, '')) + '.txt'

  f = open(stringName, "r")
  text2 = f.read()
  #print(text2)

  img = cv2.imread(imageName)
  outputs = predictor(img)


  pred_img = predictor(img)
  kotak = pred_img["instances"].pred_boxes
  x1,x2,y1,y2 = 0,0,0,0
  for i in kotak.__iter__():
    x1 = int(i.cpu().numpy()[0])
    y1 = int(i.cpu().numpy()[1])
    x2 = int(i.cpu().numpy()[2])
    y2 = int(i.cpu().numpy()[3])
    cropped_image = img[y1:y2, x1:x2]
    cropped_image = cv2.cvtColor(img[y1:y2, x1:x2], cv2.COLOR_BGR2GRAY)
    reader = easyocr.Reader(['id'])
    result = reader.readtext(cropped_image, width_ths = 10000, ycenter_ths = 0.50, slope_ths = 100
                              , allowlist = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
                              , height_ths = 250, decoder = 'wordbeamsearch', beamWidth = 10) #CUMA PLAT NOMOR



    text = ''
    if(result == []):
      text = 'null'
    else:
      text = text + result[0][1]
      text = text.upper().replace(' ', '')
      text = cek_plat(text)

    character_comparison = compare_character_counts(text2, text)
    '''
    for char, counts in character_comparison.items():
      print(f"'{char}': String1 = {counts[0]}, String2 = {counts[1]}")
    '''
    for char,counts in character_comparison.items():
      if len(text2) == len(text):
        if counts[0] == counts[1]:
          cm_tp += counts[1]
        elif counts[0] > counts[1]:
          cm_tp += counts[1]
          cm_fn += counts[0] - counts[1]
      elif len(text2) > len(text):
        if counts[0] == counts[1]:
          cm_tp += counts[1]
        elif counts[0] > counts[1]:
          cm_tp += counts[1]
          cm_fn += counts[0] - counts[1]
      elif len(text2) < len(text):
        cm_fp += len(text) - len(text2)
        if counts[0] == counts[1]:
          cm_tp += counts[1]
        elif counts[0] > counts[1]:
          cm_tp += counts[1]
          cm_fn += counts[0] - counts[1]

precision = cm_tp/(cm_tp+cm_fp)
recall = cm_tp/(cm_tp+cm_fn)
f1_score = 2 * (precision * recall) / (precision + recall)

print("True Positive : ",cm_tp)
print("False Positive : ",cm_fp)
print("False Negative : ",cm_fn)
print("True Negative : ",cm_tn)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)